In [ ]:
# Include these lines and comment out "ee.Authenticate()" after the 
# first use to avoid authentication at each run.
import ee
ee.Authenticate()
ee.Initialize()
import sys

# import all the necessary libraries
%run PlotToSat.ipynb

# By default recursion is 1000. By increasing it PloToSat can handle 
# more plot data at once but you are doing it at your own risk as
# raising it too much could cause the system to crash
sys.setrecursionlimit(10000)

# Definition of Study area
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
polygon = countries.filter(ee.Filter.eq('country_na', 'Spain'))

# Create a dictionary that holds the relevant plot information
fieldData = {
       "csvfilename"   : "./samplePlots.csv",
       "proj"          :"EPSG:3042",
       "radius"        :25,
       "xcol"          :"CX",    
       "ycol"          :"CY",
       "outPlotFileWithIDs"  :"plotsWithIDs/SpainIDs_2.csv"
}

# Specify the year of interest
year = 2020

# Create an instance of the PlotToSat class
myPlotToSat = PlotToSat(polygon,fieldData,year) 

# With this command you can see all the supported collections and the
# associated labels needed for adding them into the PlotToSat instance
myPlotToSat.printAvailableCollections()

# Example of defining the optional masks
masks = {
       "gsw": 30, 
       "lmask": 30, 
       "forestMask": {
              "buffer":30, 
              "startDate":'2000-01-01', 
              "endDate":'2019-12-31'
              }
       } 
myPlotToSat.setMasks(masks)

# GEE limits processing of data, so PloToSat divides plots data into 
# groups. The default size of a group is 400 plots. A bigger number 
# produces less files to be merged and uses less GEE requests. But if 
# it is too big GEE returns an ERROR. So some testing is required here 
# to tune the sampling size. 
myPlotToSat.setSampling(300)

# Adding Earth Observation Collections
myPlotToSat.addCollection("sentinel-1", False) 
myPlotToSat.addCollection("sentinel-2", 50  ) 

# Definition and exportation of outputs
myPlotToSat.exportFeatures("folderSpain", "r25_2020")	

# Command for re-running a subgroup of plots in case of time-out Errors
myPlotToSat.exprtFeaturesMinMax("folderSpain2","r25_2020",300,600)	